In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def plot_histogram(res):
    fig, ax = plt.subplots()
    res.plot(ax=ax, range=(-5.5, 5.5), bins=11)
    ax.set_title(v)
    ax.set_xlabel("normalized residual")
    ax.set_ylabel("number count")
    plt.show()
    plt.close()
    fig.clear()

In [ ]:
def plot_mean(res, dims: list[str], vmin):
    fig, ax = plt.subplots()
    res.mean(dim=dims).plot(
        ax=ax, cbar_kwargs={"label": "mean normalized residual"}, vmin=vmin
    )
    ax.set_title(v)
    plt.show()
    plt.close()
    fig.clear()

In [ ]:
def plot_std(res, dims: list[str], vmin, vmax):
    fig, ax = plt.subplots()
    res.std(dim=dims).plot(
        ax=ax,
        cbar_kwargs={"label": "std. dev. normalized residual"},
        vmin=vmin,
        vmax=vmax,
    )
    ax.set_title(v)
    plt.show()
    plt.close()
    fig.clear()

In [ ]:
def show_statistics(res):
    print(f"mean = {res.mean().compute().item() :.3f}")
    print(f"std  = {res.std().compute().item() :.3f}")
    print(f"min  = {res.quantile(q=0.001).compute().item() :.3f}")
    print(f"max  = {res.quantile(q=0.999).compute().item() :.3f}")

# ESA SCOPE exchange (D. J. Ford)

In [ ]:
ds = xr.open_mfdataset(
    "./Ford_et_al_UExP-FNN-U_physics_carbonatesystem_ESASCOPE_v5.*.nc",
    engine="h5netcdf",
    chunks={"time": 12},
    combine="nested",
    concat_dim="i",
).transpose("time", "i", "latitude", "longitude")

In [ ]:
ds

In [ ]:
config = {
    "fco2": {
        "uncertainty": "fco2_tot_unc",
        "coverage": 2.0,
    },
    "flux": {
        "uncertainty": "flux_unc",
        "relative": True,
        "coverage": 2.0,
    },
    "ta": {
        "uncertainty": "ta_tot_unc",
        "coverage": 2.0,
    },
    "dic": {
        "uncertainty": "dic_tot_unc",
        "coverage": 2.0,
    },
    "pH": {
        "uncertainty": "pH_tot_unc",
        "coverage": 2.0,
    },
    "saturation_aragonite": {
        "uncertainty": "saturation_aragonite_tot_unc",
        "coverage": 2.0,
    },
}

In [ ]:
for v, a in config.items():
    res = ds[v][:, 1:] - ds[v][:, 0]
    res = res / (
        ds[a["uncertainty"]][:, 0]
        / a.get("coverage", 1.0)
        * (ds[v][:, 0] if a.get("relative", False) else 1.0)
    )

    show_statistics(res)
    plot_histogram(res)
    plot_mean(res, dims=["i", "time"], vmin=-0.05)
    plot_std(res, dims=["i", "time"], vmin=0.95, vmax=1.05)

In [ ]:
ds.close()

# ESA CCI Ocean Colour

In [ ]:
ds = xr.open_mfdataset(
    "./ESACCI-OC-L3S-OC_PRODUCTS-MERGED-1M_MONTHLY_4km_GEO_PML_OCx_QAA-200001-fv6.0.*.nc",
    engine="h5netcdf",
    chunks={"lat": 1080, "lon": 1080},
    combine="nested",
    concat_dim="i",
).transpose("time", "i", "lat", "lon")

In [ ]:
ds

In [ ]:
config = {
    "Rrs_412": {
        "bias": "Rrs_412_bias",
        "rmsd": "Rrs_412_rmsd",
    },
    "Rrs_443": {
        "bias": "Rrs_443_bias",
        "rmsd": "Rrs_443_rmsd",
    },
    "Rrs_490": {
        "bias": "Rrs_490_bias",
        "rmsd": "Rrs_490_rmsd",
    },
    "Rrs_510": {
        "bias": "Rrs_510_bias",
        "rmsd": "Rrs_510_rmsd",
    },
    "Rrs_560": {
        "bias": "Rrs_560_bias",
        "rmsd": "Rrs_560_rmsd",
    },
    "Rrs_665": {
        "bias": "Rrs_665_bias",
        "rmsd": "Rrs_665_rmsd",
    },
    "adg_412": {
        "bias": "adg_412_bias",
        "rmsd": "adg_412_rmsd",
    },
    "adg_443": {
        "bias": "adg_443_bias",
        "rmsd": "adg_443_rmsd",
    },
    "adg_490": {
        "bias": "adg_490_bias",
        "rmsd": "adg_490_rmsd",
    },
    "adg_510": {
        "bias": "adg_510_bias",
        "rmsd": "adg_510_rmsd",
    },
    "adg_560": {
        "bias": "adg_560_bias",
        "rmsd": "adg_560_rmsd",
    },
    "adg_665": {
        "bias": "adg_665_bias",
        "rmsd": "adg_665_rmsd",
    },
    "aph_412": {
        "bias": "aph_412_bias",
        "rmsd": "aph_412_rmsd",
    },
    "aph_443": {
        "bias": "aph_443_bias",
        "rmsd": "aph_443_rmsd",
    },
    "aph_490": {
        "bias": "aph_490_bias",
        "rmsd": "aph_490_rmsd",
    },
    "aph_510": {
        "bias": "aph_510_bias",
        "rmsd": "aph_510_rmsd",
    },
    "aph_560": {
        "bias": "aph_560_bias",
        "rmsd": "aph_560_rmsd",
    },
    "aph_665": {
        "bias": "aph_665_bias",
        "rmsd": "aph_665_rmsd",
    },
    "kd_490": {
        "bias": "kd_490_bias",
        "rmsd": "kd_490_rmsd",
    },
    "chlor_a": {
        "bias": "chlor_a_log10_bias",
        "rmsd": "chlor_a_log10_rmsd",
    },
}

In [ ]:
for v, a in config.items():
    x = ds[v][0, 0]
    u = np.sqrt(
        np.square(ds[a["rmsd"]][0, 0]) - np.square(ds[a["bias"]][0, 0])
    )
    if v == "chlor_a":
        u = x * np.sqrt(np.exp(np.square(np.log(10.0) * u)) - 1.0)
    res = (ds[v][0, 1:] - x) / u

    show_statistics(res)
    plot_histogram(res)
    plot_mean(res, dims=["i"], vmin=-1.5)
    plot_std(res, dims=["i"], vmin=0.5, vmax=1.5)

In [ ]:
ds.close()

# GHRSST

In [ ]:
ds = xr.open_mfdataset(
    "./200001-ESACCI-L4_GHRSST-SSTdepth-OSTIA-GLOB_CDR3.0-v02.0-fv01.0-1M.*.nc",
    engine="h5netcdf",
    chunks={"lat": 1800, "lon": 3600},
    combine="nested",
    concat_dim="i",
).transpose("time", "i", "lat", "lon")

In [ ]:
ds

In [ ]:
config = {
    "analysed_sst": {
        "uncertainty": "analysed_sst_uncertainty",
        "distribution": "normal",
    }
}

In [ ]:
for v, a in config.items():
    res = (ds[v][0, 1:] - ds[v][0, 0]) / ds[a["uncertainty"]][0, 0]

    show_statistics(res)
    plot_histogram(res)
    plot_mean(res, dims=["i"], vmin=-1.5)
    plot_std(res, dims=["i"], vmin=0.5, vmax=1.5)

In [ ]:
ds.close()

# GLORYS

In [ ]:
ds = xr.open_mfdataset(
    "./mercatorglorys12v1_gl12_mean_200001.*.nc",
    engine="h5netcdf",
    mask_and_scale=True,
    chunks={"depth": 10, "latitude": 511, "lon": 1080},
    combine="nested",
    concat_dim="i",
)

In [ ]:
ds

In [ ]:
config = {
    "so": {
        "uncertainty": 0.1,
    }
}

In [ ]:
for v, a in config.items():
    res = (ds[v][1:, 0, 0] - ds[v][0, 0, 0]) / a["uncertainty"]

    show_statistics(res)
    plot_histogram(res)
    plot_mean(res, dims=["i"], vmin=-1.5)
    plot_std(res, dims=["i"], vmin=0.5, vmax=1.5)

In [ ]:
ds.close()